# Vegetation Change Detection


- **Special requirements:** A Google account, access to Google Earth Engine.

- **Prerequisites:** You should have completed the "Week 4 - Prac 1" and "Week 5 - Prac 1" notebooks.

## Description

In this practical we will conduct change detection using the band indice NDVI to identify area burnt during the Namadgi bush fires of 2020. 

## Aims of the practical session
* Create image composites from before and after the bush fire event
* Calculate NDVI on both image compsites
* Calculate the difference in NDVI between events

## Load packages

Import GEE packages that are needed for the analysis.

In [ ]:
import ee
import geemap
# ee.Authenticate()

## Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [ ]:
Map = geemap.Map()
Map.addLayerControl()

## Loading Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map. In this example, we'll import a geojson from github that marks the boundary of Namadagi National Park region.

In [ ]:
path = '../figures/namadgi_region.geojson'
geometry = geemap.geojson_to_ee(path)
Map.addLayer(geometry, {}, 'Namadgi Region')
Map.centerObject(geometry);

### Load Landsat 8 and calculate an image composite from 2018 

In the next few code cells we will extract training data from Landsat 8 images over the region specified by the ROI

We will:
* Define a function for cloud masking and rescaling Landsat 8 images
* Load Landsat 8 images for the analysis
* Filter a collection by date range
* Calculate a temporal median to collapse the time dimension
* Clip based on the geometry

In [ ]:
def rescale_landsatC2(image):
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True)

def cloudMaskL8Collection2(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

  # Replace the original bands with the scaled ones and apply the masks.
    return image.updateMask(qaMask) \
          .updateMask(saturationMask)

In [ ]:
## collect before event images 
ls8_before = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(geometry)
    .filterDate('2018-01-01', '2018-12-31') 
    .map(rescale_landsatC2) #map the cloudmasking/rescaling function
    .map(cloudMaskL8Collection2)
    .median() #temporal composite
    .clip(geometry)
)

### Visualize the 2018 Landsat Composite

In [ ]:
vis_params = {'min': 0, 'max': 0.4, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}

#Map.centerObject(point, 8)
Map.addLayer(ls8_before, vis_params, "Landsat8_before")
Map.addLayerControl()
Map

## Calculate NDVI for 2018

We can use the earth engine function `normalizedDifference()`, we just need to pass it the `NIR` and `Red` bands. Refer to your lecture notes for the theory on NDVI

In [ ]:
ndvi_before = ls8_before.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI_before')

### Add the NDVI image to the map

In [ ]:
#define a pretty palette for NDVI
ndvi_palette=['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

Map.addLayer(ndvi_before, {'min': 0, 'max': 1, 'palette': ndvi_palette}, 'NDVI Before');
Map

## Load Landsat 8 and calculate an image composite from 2020 

In the next few code cells we will extract training data from Landsat 8 images over the region specified by the ROI

We will:
* Define a function for cloud masking and rescaling Landsat 8 images
* Load Landsat 8 images for the analysis
* Filter a collection by date range
* Calculate a temporal median to collapse the time dimension
* Clip based on the geometry

In [ ]:
# # After Images.
ls8_after = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(geometry)
    .filterDate('2020-01-01', '2020-12-31')
    .map(rescale_landsatC2) #map the cloudmasking/rescaling function
    .map(cloudMaskL8Collection2)
    .median() #temporal composite
    .clip(geometry)
)

In [ ]:
Map.addLayer(ls8_after, vis_params, "Landsat8_after")
Map.addLayerControl()
Map

## Calculate NDVI for 2020

In [ ]:
ndvi_after = ls8_after.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI_after')

Map.addLayer(ndvi_after, {'min': 0, 'max': 1, 'palette': ndvi_palette}, 'NDVI After Event');
Map

## Create NDVI Difference Map

To identify areas of change, we will difference (subtract) the two NDVI maps. Areas that have undergone large negative change (lower NDVI in 2020 compared with 2018) will be highlighted in red, while regions that are greener than 2018 will display in blue. Areas that have seen little change will be white(ish)

In [ ]:
# # # Create NDVI Difference Map
diff = ndvi_before.subtract(ndvi_after);

In [ ]:
Map.addLayer(diff,
            {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']},
            'NDVI Difference');
Map

<div class="alert alert-block alert-danger"> 

## Exercise 1 - NDVI change detection

1. Which areas underwent a reduction in NDVI between 2018 and 2020, and why? Which areas saw a postive increase in NDVI, and why?
2. Create a new map that thresholds the `diff` map to isolate areas that have seen a large change in NDVI due to the fires (ie. not a minor fluctuation). e.g You might use a theshold such as -0.2, where any area that saw a NDVI change larger than -0.2 would count as 'significant' change. You can test different thresholds to determine one that best delineates the burnt areas. The final map will be a binary map of 0 and 1s, where pixels labelled as 1 will be areas where the NDVI difference is 'significant', and 0 where the change in 'non-significant'.
3. Calculate the burnt area (in square kilometers) using the thresholded difference map you just created.
4. What other band indices might be useful for delineating the burnt area?  
    
<div>

Your code goes here:

<div class="alert alert-block alert-danger"> 

## Challenge

Create a map of burnt area over Namadgi using one of the alternative band indices you identified in question 4 of Exercise 1. Calculate the area and compare it with the area you calculated using NDVI.
    
<div>

## References
This is where the references go. For exmaple:

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
* "Earth Observation: Data, Processing and Applications" book. Available through Wattle, or http://www.crcsi.com.au/earth-observation-series.

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** August 2022